In [1]:
import tensorflow

In [ ]:
import zipfile

zip_path =r"C:\Users\saran\Desktop\XRAY\chest_xray-20250726T110654Z-1-001.zip"
extract_path = "dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Done!")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
# setting the training data directory
datadir=r"dataset/chest_xray/train/"

In [ ]:
# initializing the category of classification
categories= ['normal','pneumonia']

In [ ]:
! pwd

In [ ]:
! ls

In [ ]:
! ls dataset/chest_xray/test

In [ ]:
# converting the images to grayscale 
for i in categories: 
    path= os.path.join(datadir,i) 
    for img in os.listdir(path): 
        img_array= cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) 
        plt.imshow(img_array, cmap='gray') 
        plt.show() 
        break 
    break

In [ ]:
# initializing the size of the image so that every image should be in same direction
img_size=100

In [ ]:
# image after resize and grayscale
new_array = cv2.resize(img_array,(img_size,img_size))
plt.imshow(new_array, cmap='gray')
plt.show()

In [ ]:
training_data=[]

In [ ]:
# defining the funstion for training data according to the categories converting he images into gray scale
# converting it into numpy arrays
def create_training_data():
    for i in categories:
        path= os.path.join(datadir,i)
        class_num= categories.index(i)

        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array,(img_size,img_size))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

In [ ]:
create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample)

In [ ]:
# splitting the features and labels
x=[]
y=[]
for features,label in training_data:
    x.append(features)
    y.append(label)

In [ ]:
print(x[0].reshape(-1,img_size,img_size,1))

In [ ]:
y=np.array(y)

In [ ]:
# reshaping the features for making it compatible with tensorflow
x= np.array(x).reshape(-1,img_size,img_size,1)

In [ ]:
# as we do for training data similarlt doing with validation data
validation_data=[]
datadir_val=r"dataset/chest_xray/val/"

In [ ]:
def create_validation_data():
    for i in categories:
        path= os.path.join(datadir_val,i)
        class_num= categories.index(i)

        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array,(img_size,img_size))
                validation_data.append([new_array,class_num])
            except Exception as e:
                pass

In [ ]:
create_validation_data()

In [ ]:
print(len(validation_data))

In [ ]:
import random
random.shuffle(validation_data)
for sample in validation_data[:10]:
    print(sample)

In [ ]:
x_val=[]
y_val=[]

In [ ]:
for features,label in validation_data:
    x_val.append(features)
    y_val.append(label)

In [ ]:
x_val=np.array(x_val).reshape(-1,img_size,img_size,1)
y_val=np.array(y_val)

In [ ]:
#importing the library for training the model and adding the neural networj layers
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation, Flatten
import pickle

In [ ]:
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
x=x/255.0
x_val=x_val/255.0 
# this is feature scaling step (in order to scale it down from 0 to 1)


In [ ]:
x.shape

In [ ]:
# model the neural network layer for training the model
model= Sequential()

model.add(Conv2D(64,(3,3),input_shape=x.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Conv2D(256,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(64))

model.add(Dropout(.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss= 'binary_crossentropy', optimizer ='adam', metrics=['accuracy'])

In [ ]:
# fitting the model with 10 epochs and using the validation data we have created
model.fit(x,y,batch_size=4, epochs=10, validation_data= (x_val,y_val))

In [ ]:
model.save(r'model_10.h5')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

In [ ]:
# function for preparing the image for predicting the class
def prepare(image):
    img_size=100
    # img_array= cv2.imread(image,cv2.IMREAD_GRAYSCALE)
    img= tf.keras.preprocessing.image.load_img(image, color_mode='grayscale', target_size=(img_size,img_size))
    new_array=tf.keras.preprocessing.image.img_to_array(img)

    return new_array.reshape(-1,img_size,img_size,1)

In [ ]:
model=tf.keras.models.load_model(r'model_10.h5')


In [ ]:
image = r"C:\Users\saran\Desktop\ML\X_ray\dataset\chest_xray\test\PNEUMONIA\person16_virus_47.jpeg"


In [ ]:
prediction = model.predict([prepare(image)/255.0])
print(prediction)
print(round(prediction[0][0]))
print(categories[int(round(prediction[0][0]))])

In [ ]:
test_dir = r'C:\Users\saran\Desktop\XRAY\dataset\chest_xray\test'
model =load_model(r"C:\Users\saran\Desktop\ML\X_ray\model_10.h5")
x_test = []
y_test = []

# Loop through the test data directory and extract the images and their labels
for category in categories:
    path = os.path.join(test_dir,category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (img_size, img_size))
            x_test.append(new_array)
            y_test.append(class_num)
        except Exception as e:
            pass

# convert test data to numpy arrays
x_test = np.array(x_test).reshape(-1,img_size,img_size,1)
y_test= np.array(y_test)

#normalize test data
x_test =x_test/255.0

# calculate test accuracy
test_loss, test_acc =model.evaluate(x_test,y_test, verbose=2)

print('Test accuracy:', test_acc)

## Pre Trained Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#DEFINE IMAGE SIZE
img_size=100

# load the pre_trained model (VGG16)
base_model = VGG16(input_shape = (img_size,img_size,3), include_top= False, weights = 'imagenet')

# freeze the layers of the pre trained model
for layer in  base_model.layers:
    layer.trainable= False

# add custom layers for classification
x= Flatten()(base_model.output)
x= Dense(256, activation='relu')(x)
x= Dense(128, activation='relu')(x)
x= Dense(64, activation='relu')(x)
predictions= Dense(1, activation= 'sigmoid')(x)

#create a new model 
model= Model(inputs= base_model.input, outputs= predictions)

#compile the model
model.compile(optimizer='adam', loss= 'binary_crossentropy' , metrics=['accuracy'])

# define the image generators for training and validation data
train_datagen= ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip= True)
val_datagen= ImageDataGenerator(rescale = 1./255)

#specify the training and validation data directories
train_dir = r"C:\Users\saran\Desktop\ML\X_ray\dataset\chest_xray\train"
val_dir = r"C:\Users\saran\Desktop\ML\X_ray\dataset\chest_xray\val"

#create the image generators for training and validation data
train_generator= train_datagen.flow_from_directory(train_dir, target_size=(img_size,img_size), batch_size=32, class_mode='binary')
val_generator= val_datagen.flow_from_directory(val_dir, target_size=(img_size,img_size), batch_size=32, class_mode='binary')

#train the model
model.fit(train_generator, epochs=10, validation_data= val_generator)

# evaluate the model on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator =test_datagen.flow_from_directory(test_dir, target_size=(img_size,img_size), batch_size=32, class_mode='binary', shuffle=False)
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


In [ ]:
# here it is clear that we are getting better accuracy than our model.